<a href="https://colab.research.google.com/github/ravi-prakash1907/Data-Structures-and-Algo/blob/main/Submissions/Group%20Projects/Project3/English_Sentence_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3**

## English Sentence Generator

<!--
_**([Final Report Here!!](#))**_
-->


### _Libraries_

In [ ]:
from IPython.display import clear_output
import re

## Input Text Processing

**Steps:**  
1. Get Sentance (Input)  
2. Convert to Lower/Upper Case  
3. Resolve for Abbriviation (ain't, I've, can't...)  
4. Remove Negation  
5. Resolve for Interrogative  
6. Remove Stopping Symbol (. ! ? : ; etc..)
7. Split the Sentance and Store in Dictionary  
  - _Parts of Sentance:_  
    a) Subject  
    b) Helping Verb  
    c) Verb + Object  

_(we can later split **b** in 2-levels and **c** in 2 saperate parts)_

In [ ]:
## List of words to sentance
def listToSentance(buildingBlocks):
  newSentance = ""
  for word in buildingBlocks:
    newSentance += word + " "
  
  return newSentance[:-1]   # last letter is space (from for-loop)  

### Processing Level 1

1. Taking Input  
2. Converting to Lower Case  
3. Resolving for Abbriviation  
4. Removing Negation  

_(Step 1, 2, 3 and 4)_

In [116]:
## part of text data cleaner
def toLower(sentance):
  return sentance.lower()


# resolving for abbreviations
def resolveAbbr(sentance):
  if '\'' in sentance:
    splittedSentance = sentance.split(" ")

    for index in range(0, len(splittedSentance)):
      word = splittedSentance[index]

      # abbriviated 'not'
      if word[-3:] == 'n\'t':
        if word == 'can\'t':
          word = 'can not'
        elif word == 'ain\'t':
          word = 'are not'  # or 'is not' too in some cases still it'll work for now
        elif word == 'won\'t':
          word = 'will not'
        elif word == 'shan\'t':
          word = 'shall not'
        else:
          word = word[:-3] + " not"
      
      # abbriviated 'ing'
      if word[-3:] == 'in\'':
        word = word[:-1] + 'g'
      
      # abbriviated helping verbs
      abbrHVerbs = {'m':'am', 's':'is', 're':'are', 'll':'will', 've':'have', 'd':'had'}
      if "'" in word:
        flag = False
        brokenWord = word.split('\'')
        for abbr in list(abbrHVerbs.keys()):
          if  abbr == brokenWord[1]:
            brokenWord[1] = abbrHVerbs[abbr]
            flag = True

        if flag:
          word = listToSentance(brokenWord)
        else:
          word = brokenWord[0] + '\'' + brokenWord[1]
      
      splittedSentance[index] = word

    resolvedTense = listToSentance(splittedSentance)
    return resolvedTense

  return sentance


## Later we'll read sntances from some other source(s).
def readSentance():
  textData = input("Enter a sentance: ")
  return textData


## Removing Negation
def negationRemover(sentance):
  if 'not' in sentance:
    begIndex = sentance.find("not")
    sentance = sentance[:begIndex-1] + sentance[begIndex+3:]
    sentance = negationRemover(sentance)
  return sentance


### Processing Level 2

1. Resolving for Interrogatives  
2. Remove Stopping Symbol (. ! ? : ; etc..)

_(Step 5)_

In [119]:
## Yet to be implemented
def removeQuestion(sentance):
  print("Interrogative to normal sentance converter is yet to be impleented!\n")
  return sentance

def removeStops(sentance):
  if not sentance[-1].isalpha() and not sentance[-1].isdigit():
    sentance = removeStops(sentance[:-1])
  return sentance

#### **Driver (for testing) Function**

In [120]:
def driverFun():
  aSentance = readSentance()
  print()
  
  oneCase = toLower(aSentance)
  abbrResolved = resolveAbbr(oneCase)
  positiveTense = negationRemover(abbrResolved)
  nonQuestion = removeQuestion(positiveTense)
  cleanSentance = removeStops(nonQuestion)
  print(cleanSentance)


##############

driverFun()

##  "He isn't a bad boy, although he's not ready for singin' for now as it's already 10 O'clock!"

Enter a sentance: He isn't a bad boy, although he's not ready for singin' for now as it's already 10 O'clock!

Interrogative to normal sentance converter is yet to be impleented!

he is a bad boy, although he is ready for singing for now as it is already 10 o'clock


### Processing Level 3

**Initial Splitting** _(if consists Helping Verb)_  

**Parts of Sentance:**  
  - Subject + Helping Verb  
  - Verb + Object  


_(Step 6.1)_

In [139]:
## Building Blocks

# splits as:  ['sub + HV', 'verb + obj']
def splitOnHelpingVerb(sentance):
  ## H.V. to split
  firstPriority = ['am', 'is', 'are', 'was', 'were']
  secondPriority = ['been', 'be']
  thirdPriority = ['have']
  forthPriority = ['can', 'has', 'shall', 'will', 'do', 'does', 'did', 'should', 'may', 'might', 'would' 'must', 'could', 'had']

  helpingVerbs = [firstPriority, secondPriority, thirdPriority, forthPriority]
  initSplit = []
  splitted = False
  
  for priorityList in helpingVerbs:
    for helpingVerb in priorityList:
      if helpingVerb in sentance:
        initSplit = sentance.split(helpingVerb)
        initSplit[0] += helpingVerb
        splitted = True
        break
    if splitted:
      break
  
  return initSplit


def consistsHelpingVerb(sentance = "he goes"):
  ## Helping Verbs List
  helpingVerbs = ['am', 'is', 'are', 'was', 'were', 'been', 'be', 'can', 'has', 'shall', 'will', 'do', 'does', 'did', 'have', 'should', 'may', 'might', 'would' 'must', 'could', 'had']
  for HV in helpingVerbs:
    if HV in sentance:
      return splitOnHelpingVerb(sentance)
  return False

#######################################

## he is a bad boy, although he is ready for singing for now as it is already 10 o'clock
consistsHelpingVerb()


False

### Processing Level 4

**Secondary Splitting**  

_Parts of **Subject + Helping Verb** :_  
  - Subject  
  - Helping Verb  

_Parts of **Verb + Object** :_  
  - Verb  
  - Object  


_(Step 6.2 & 6.3)_

In [136]:
## split ["Sub", "helpingVerb / verb / verb+obj"]
def splitFromBeg(sentance, consistsHV = False):
  wordList = sentance.split(" ")
  if len(wordList) == 2:
    return wordList
  elif len(wordList) > 2 and consistsHV:
    firstPriority = ['am', 'is', 'are', 'was', 'were']
    secondPriority = ['been', 'be']
    thirdPriority = ['have']
    forthPriority = ['can', 'has', 'shall', 'will', 'do', 'does', 'did', 'should', 'may', 'might', 'would' 'must', 'could', 'had']
    
    helpingVerbs = [firstPriority, forthPriority, thirdPriority, secondPriority]
    begSplitted = []
    splitted = False
    
    for priorityList in helpingVerbs:
      for helpingVerb in priorityList:
        if helpingVerb in sentance:
          begSplitted = sentance.split(helpingVerb)
          #HVBegIndex = sentance.find(helpingVerb)
          begSplitted[1] = helpingVerb + begSplitted[1]
          splitted = True
          break
      if splitted:
        break
    return begSplitted
  else:
    print("Yet to be implemented for sentences without HV. (like: 'he goes.')")

In [138]:
sentance = ['He have been eating', 'a bad boy']
splitFromBeg(sentance[0], True)

['He ', 'have been eating']

## Storing in Tree

**Steps:**  

# **Users' Zone**

## Main-Menu

In [ ]:
def menu():
  # Printing Menu
  print("\nMain Menu:")
  print("""----------
        \n1. Load Raw Images  \n2. Create New Album  \n3. Grab n-th Image \n4. Print Loaded Images \n5. Print Newly Created Album \n
        \n** Any other key to exit!!\n
        """)
  return input("Enter your choice: ")

## Main-Function


In [ ]:
### To be Implemented

if __name__ == "__main__":
  
  ## Showing Menu

  while (True): 
    clear_output()
    #choice = menu()
    
    print("Yet to be implemented!\nMeet Soon!!!")
    break

Yet to be implemented!
Meet Soon!!!
